In [1]:
import requests
import json
import pandas as pd
import numpy as np 

from datetime import datetime, timedelta, timezone
from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt

import psychrolib # Library for WetBulb estimation 

In [2]:
from utils.data_preprocess import convert_str_double, fix_nan_value, detect_missing_values
from utils.data_preprocess import convert_to_TWetBulb
from utils.data_preprocess import round_hour, align_time, 
from utils.data_preprocess import repair_single_fuel

## 1. Weather Data Preprocessing

In [4]:
# 4S2_Oregan_NW.csv    HND_Nevada_CAL.csv   JYO_virginia_PJM.csv
# FTY_Georgia_SE.csv   JWY_Texas_ERCO.csv

file_name = "JWY_Texas_ERCO"
weath_path = "data/weather_raw/{}.csv".format(file_name)
df = pd.read_csv(weath_path)

###### Align time into hours
df = align_time(df)
df = df.drop_duplicates(subset=['valid'])
if file_name in ["JWY_Texas_ERCO", "JYO_virginia_PJM"]:
    df = df.iloc[1:,:]

####    Data Cleaning, remove "M" values       ######

data_dict = {"station": df["station"],
             "valid"  : df["valid"]   }

# Convert the temperature string list to np.double array
selected_cols = ["tmpf", "dwpf", "relh"]
for k in selected_cols:
    temperature_value = df[k].values
    arr = convert_str_double(temperature_value)
    data_dict[k] = arr

# Convert the pressure list to np.double array 
pressure = df["mslp"].values
if np.all(pressure=="M"):
    # The typical pressure at sea level is 1013.25 millibars or 14.7 pounds per square inch (PSi).
    arr = np.ones(pressure.shape)*1013.25
else:
    arr = convert_str_double(pressure)
data_dict["mslp"] = arr

# Convert the results to a dataframe 
df_res = pd.DataFrame(data_dict)
missed_array = detect_missing_values(df_res["valid"].values)

# Create a data template
dummy_head_i = df_res.iloc[0,:].copy()
selected_cols = ["tmpf", "dwpf", "relh", "mslp"]

# Set all the values in this dummy row as NaN
for k in selected_cols:
    dummy_head_i[k] = np.nan

# Insert all the missed values
for missed_key in missed_array:
    dummy_head_i["valid"]     = missed_key
    df_res = df_res.append(dummy_head_i)

# Only select 1-year data and sort based on time stamp
df_res = df_res.sort_values(["valid"])
df_res = df_res.iloc[:8759,:]  


# Fix the NaN values with linear interpolation
for k in selected_cols:
    repaired_arr = fix_nan_value(df_res[k])
    df_res[k]    = repaired_arr

In [5]:
###### Calculate the wet bulb temperature
drytemp      = df_res["tmpf"].values
relh         = df_res["relh"].values
pressure     = df_res["mslp"].values

wettemp      = convert_to_TWetBulb(drytemp, relh, pressure)
df_res.insert(4, 'wbtmp',wettemp)

In [8]:
df_res
# weath_path_out = "data/weather/{}.csv".format(file_name)
# df_res.to_csv(weath_path_out, index=False, float_format='%.2f')

,station,valid,tmpf,dwpf,wbtmp,relh,mslp
1,JWY,2022-01-01T01,67.8,63.1,64.690061,84.91,1013.25
4,JWY,2022-01-01T02,67.3,63.5,64.779220,87.61,1013.25
7,JWY,2022-01-01T03,66.6,63.0,64.224789,88.19,1013.25
10,JWY,2022-01-01T04,67.1,63.7,64.841166,88.84,1013.25
13,JWY,2022-01-01T05,69.3,65.7,66.858357,88.32,1013.25
...,...,...,...,...,...,...,...
26191,JWY,2022-12-31T19,66.0,47.3,55.370966,50.86,1013.25
26194,JWY,2022-12-31T20,69.6,45.7,56.086984,42.29,1013.25
26197,JWY,2022-12-31T21,68.9,45.7,55.811241,43.32,1013.25
26200,JWY,2022-12-31T22,72.9,46.6,57.757071,39.12,1013.25


## 2. EIA Data Cleaning

In [8]:
coorp_name = "ERCO"

fuel_mix_path = "data/fuelmix_raw/{}_year_2022.csv".format(coorp_name)
num_timeslot  = 24*60

df_raw = pd.read_csv(fuel_mix_path).drop("Unnamed: 0", axis=1)

# Create data template the the dataframe 
dummy_head = df_raw.iloc[0,:].copy()
dummy_head["value"]     = np.nan
dummy_head["fueltype"]  = "NA"
dummy_head["type-name"] = "NA"

# Create dataframe list
df_list = []
fuel_list = list(set(df_raw["fueltype"].values))

for fuel_name in fuel_list:
    
    df_i = df_raw[df_raw["fueltype"]==fuel_name]
    df_i = repair_single_fuel(df_i, dummy_head)

    value_inter = fix_nan_value(df_i["value"].values)
    df_i = df_i.assign(value=value_inter)
    
    df_list.append(df_i)

df_res = pd.concat(df_list)
df_res = df_res.sort_values(['period', "fueltype"])
df_res = df_res.iloc[8:,:]  
# For Texas, this line should be 
# df_res = df_res.iloc[7:,:] 

fuel_mix_out = "data/fuelmix/{}_year_2022.csv".format(coorp_name)
df_res.to_csv(fuel_mix_out, float_format='%.2f', index=False)